<details>
<summary>Table of Contents</summary>

- [1. Standard Scaler Informer ](#1-standard-scaler-informer)
- [2. Standard Scaler PatchTST](#2-standard-scaler-patchtst)
- [3. MinMax (0, 1) Scaler Informer](#3-minmax-scaler-0-1-informer)
- [4. MinMax (0, 1) Scaler PatchTST](#4-minmax-scaler-0-1-patchtst)
- [5. MinMax (0, 5) Scaler Informer](#5-minmax-scaler-0-5-informer)
- [6. MinMax (0, 5) Scaler PatchTST](#6-minmax-scaler-0-5-patchtst)

</details>


Here we perform a check on DE dataset to confirm choice of loss function and scaler for our data.

This script is to run the models. Final results are in the notebook "Comparison". 

Please note, the cell content is almost identical. However, when duplicating code and changing some arguments, it becomes easier to store and read results (especially if you want to experiment with 1 subpart) and split long running time into subprocesses. 

In [1]:
import os
import pandas as pd
import subprocess
import shutil
from utils.helper import extract_metrics_from_output, convert_results_into_df

# 1. Standard Scaler Informer

In [2]:
# Paths to files and data
data_path = os.getcwd() + "/datasets/"

script_path = "./PatchTST-main/PatchTST_supervised/run_longExp.py"

# Arguments that will be used also for file names
model = "Informer"
dataset = 'IT_data.csv'
losses = ["MSE", "RMSE", "MAE"]
country = dataset[:2]

log_dir = f"logs/loss_choice/standard"
if not os.path.exists(log_dir):
    os.makedirs(log_dir)

log_file_path = f"{log_dir}/{model}_{country}.log"

In [4]:
# Dynamic variables
pred_lens = ["24", "96", "168"]
seq_len = "96"
lr = "0.0001"
itr = 2  
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

# Lists to store the results
informer_results_scaled, informer_results_unscaled = [], []

# Log file
with open(log_file_path, "w") as log_file:
    for loss in losses:
        statement_1 = f"\n=== Starting experiments for loss function: {loss} ===\n"
        log_file.write(statement_1)
        print(statement_1)  # Print to .ipynb output cell

        for pred_len in pred_lens:
            statement_2 = f"\n=== Starting experiments for pred_len: {pred_len} ===\n"
            log_file.write(statement_2)
            print(statement_2) 
            model_id = f"{country}_{seq_len}_{pred_len}_loss_choice_for_{country}"

            # Arguments for the command
            command = f"""
            python {script_path} \
              --random_seed 2021 \
              --is_training 1 \
              --root_path "{data_path}" \
              --data_path "{dataset}" \
              --model_id {model_id} \
              --model "{model}" \
              --data "custom" \
              --features M \
              --seq_len {seq_len} \
              --label_len 5 \
              --pred_len {pred_len} \
              --e_layers 2 \
              --d_layers 1 \
              --factor 5 \
              --enc_in 3 \
              --dec_in 3 \
              --c_out 3 \
              --des 'Exp' \
              --train_epochs 10 \
              --patience 3 \
              --overlapping_windows \
              --inverse \
              --loss_fnc "{loss}" \
              --scaler_type standard \
              --itr {itr} --batch_size 32 --learning_rate "{lr}"
            """

            # Run the command and capture the output
            process = subprocess.Popen(command, shell=True, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True)

            # Capture the output in real-time
            output = []
            for line in process.stdout:
                output.append(line)
                print(line, end='')  # Print in the .ipynb cell
                log_file.write(line)  # Write to the log file

            # Wait for the process to complete
            process.wait()
            
            # Delete the checkpoints folder and all its contents
            shutil.rmtree('./checkpoints' )

            # Extract metrics for each iteration
            iteration_metrics = extract_metrics_from_output(output, itr)
            iteration_metrics_unscaled = extract_metrics_from_output(output, itr, if_scaled=False)

            # Log the extracted metrics and save them
            for iteration, (scaled_metrics, unscaled_metrics) in enumerate(zip(iteration_metrics, iteration_metrics_unscaled), start=1):
                log_file.write(f"\nExtracted Metrics for {country}, pred_len={pred_len}, iteration={iteration}:\n")
                log_file.write(f"Scaled Metrics - MSE: {scaled_metrics[0]}, RMSE: {scaled_metrics[1]}, MAE: {scaled_metrics[2]}, RSE: {scaled_metrics[3]}\n")
                log_file.write(f"Unscaled Metrics - MSE: {unscaled_metrics[0]}, RMSE: {unscaled_metrics[1]}, MAE: {unscaled_metrics[2]}, RSE: {unscaled_metrics[3]}\n")

                # Append the results to the informer_results lists
                metrics_data = [(informer_results_scaled, scaled_metrics), (informer_results_unscaled, unscaled_metrics)]

                for result_list, metrics in metrics_data:
                    result_list.append({
                        'Loss_function': loss,
                        'Pred_len': pred_len,
                        'Iteration': iteration,
                        'MSE': metrics[0],
                        'RMSE': metrics[1],
                        'MAE': metrics[2],
                        'RSE': metrics[3]
                    })


=== Starting experiments for loss function: MSE ===


=== Starting experiments for pred_len: 24 ===

Args in experiment:
Namespace(random_seed=2021, is_training=1, model_id='IT_96_24_loss_choice_for_IT', model='Informer', data='custom', root_path='/vol/fob-vol3/nebenf24/riabchuv/my_work/datasets/', data_path='IT_data.csv', features='M', target='OT', freq='h', checkpoints='./checkpoints/', overlapping_windows=True, scaler_type='standard', seq_len=96, label_len=5, pred_len=24, inverse=True, loss_fnc='MSE', if_relu=False, fc_dropout=0.05, head_dropout=0.0, patch_len=16, stride=8, padding_patch='end', revin=1, affine=0, subtract_last=0, decomposition=0, kernel_size=25, individual=0, embed_type=0, enc_in=3, dec_in=3, c_out=3, d_model=512, n_heads=8, e_layers=2, d_layers=1, d_ff=2048, moving_avg=25, factor=5, distil=True, dropout=0.05, embed='timeF', activation='gelu', output_attention=False, do_predict=False, num_workers=10, itr=2, train_epochs=10, batch_size=32, patience=3, learning_rate=

In [5]:
path_dir = './dataset_results'
csv_name_scaled = 'informer_loss_functions_results_scaled_IT.csv'
csv_name_unscaled = 'informer_loss_functions_results_unscaled_IT.csv'

# Convert the results into a DataFrame and save as CSV
informer_df_scaled = convert_results_into_df(informer_results_scaled, path_dir, csv_name_scaled)
informer_df_unscaled = convert_results_into_df(informer_results_unscaled, path_dir, csv_name_unscaled)
informer_df_scaled.round(4)

MSE    RMSE     MAE     RSE
Loss_function Iteration Pred_len                                
MSE           1         24        0.2209  0.4700  0.2896  0.4304
              2         24        0.2295  0.4790  0.2907  0.4387
              1         96        0.3704  0.6086  0.4019  0.5572
              2         96        0.3623  0.6019  0.4003  0.5511
              1         168       0.4221  0.6497  0.4368  0.5950
              2         168       0.3872  0.6222  0.4291  0.5699
RMSE          1         24        0.2177  0.4665  0.2848  0.4273
              2         24        0.2264  0.4758  0.2886  0.4358
              1         96        0.3753  0.6126  0.4051  0.5609
              2         96        0.3687  0.6072  0.4023  0.5560
              1         168       0.4266  0.6532  0.4384  0.5982
              2         168       0.3803  0.6167  0.4220  0.5648
MAE           1         24        0.2191  0.4681  0.2784  0.4287
              2         24        0.2304  0.4800  0.2763  0.4396
              1         96        0.4146  0.6439  0.3969  0.5895
              2         96        0.4017  0.6338  0.3855  0.5803
              1         168       0.4431  0.6656  0.4272  0.6097
              2         168       0.4478  0.6692  0.4227  0.6129

In [6]:
informer_df_unscaled.round(4)

MSE       RMSE        MAE     RSE
Loss_function Iteration Pred_len                                           
MSE           1         24        1701676.125  1304.4830   847.7084  0.0917
              2         24        1636712.625  1279.3407   840.6756  0.0899
              1         96        3217915.500  1793.8550  1224.6313  0.1262
              2         96        3257407.000  1804.8289  1229.3171  0.1270
              1         168       4621935.500  2149.8687  1406.2321  0.1514
              2         168       4015123.000  2003.7772  1363.0995  0.1411
RMSE          1         24        1623739.375  1274.2604   826.7006  0.0895
              2         24        1724387.875  1313.1595   853.6309  0.0923
              1         96        3351209.750  1830.6310  1242.9313  0.1288
              2         96        3368993.750  1835.4819  1239.2218  0.1292
              1         168       4557453.000  2134.8191  1400.7711  0.1504
              2         168       3870673.500  1967.4027  1330.7379  0.1386
MAE           1         24        1594660.125  1262.7986   805.4221  0.0887
              2         24        1614850.000  1270.7675   776.7831  0.0893
              1         96        3466446.000  1861.8394  1189.1581  0.1310
              2         96        2930002.250  1711.7250  1105.7059  0.1205
              1         168       4373828.500  2091.3701  1345.8411  0.1473
              2         168       4235910.000  2058.1326  1310.9803  0.1450

In [7]:
# Uncomment the following lines if you want to read saved results
#path_dir = './dataset_results'
#csv_name_scaled = 'informer_loss_functions_results_scaled.csv'
#csv_name_unscaled = 'informer_loss_functions_results_unscaled.csv'

# Average the iterations
informer_scaled = pd.read_csv(os.path.join(path_dir, csv_name_scaled))
informer_unscaled = pd.read_csv(os.path.join(path_dir, csv_name_unscaled))

inf_res_scaled = informer_scaled.groupby(['Pred_len', 'Loss_function']).mean().drop('Iteration', axis=1)
inf_res_unscaled = informer_unscaled.groupby(['Pred_len', 'Loss_function']).mean().sort_index().drop('Iteration', axis=1)
inf_res_scaled.round(4)

MSE    RMSE     MAE     RSE
Pred_len Loss_function                                
24       MAE            0.2247  0.4740  0.2773  0.4341
         MSE            0.2252  0.4745  0.2901  0.4346
         RMSE           0.2220  0.4712  0.2867  0.4315
96       MAE            0.4081  0.6388  0.3912  0.5849
         MSE            0.3663  0.6053  0.4011  0.5542
         RMSE           0.3720  0.6099  0.4037  0.5584
168      MAE            0.4454  0.6674  0.4250  0.6113
         MSE            0.4046  0.6360  0.4329  0.5825
         RMSE           0.4035  0.6349  0.4302  0.5815

In [8]:
inf_res_unscaled.round(4)

MSE       RMSE        MAE     RSE
Pred_len Loss_function                                            
24       MAE            1.604755e+06  1266.7830   791.1026  0.0890
         MSE            1.669194e+06  1291.9119   844.1920  0.0908
         RMSE           1.674064e+06  1293.7100   840.1658  0.0909
96       MAE            3.198224e+06  1786.7822  1147.4320  0.1257
         MSE            3.237661e+06  1799.3419  1226.9742  0.1266
         RMSE           3.360102e+06  1833.0565  1241.0765  0.1290
168      MAE            4.304869e+06  2074.7513  1328.4107  0.1461
         MSE            4.318529e+06  2076.8229  1384.6658  0.1463
         RMSE           4.214063e+06  2051.1109  1365.7545  0.1445

# 2. Standard Scaler PatchTST

In [9]:
# Dynamic variables
pred_lens = ["24", "96", "168"]
seq_len = "512"
lr = "0.0001"
model = "PatchTST"
itr = 2  
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

# New log file path
log_file_path = f"{log_dir}/{model}_{country}.log"

patchtst_results_scaled, patchtst_results_unscaled = [], []

with open(log_file_path, "w") as log_file:
    for loss in losses:
        statement_1 = f"\n=== Starting experiments for loss function: {loss} ===\n"
        log_file.write(statement_1)
        print(statement_1)  # Print to notebook

        for pred_len in pred_lens:
            statement_2 = f"\n=== Starting experiments for pred_len: {pred_len} ===\n"
            log_file.write(statement_2)
            print(statement_2) 
            model_id = f"{country}_{seq_len}_{pred_len}_loss_choice_for_{country}"

            # Command arguments
            command = f"""
            python {script_path} \
              --random_seed 2021 \
              --is_training 1 \
              --root_path "{data_path}" \
              --data_path "{dataset}" \
              --model_id {model_id} \
              --model "{model}" \
              --data "custom" \
              --features M \
              --seq_len {seq_len} \
              --label_len 5 \
              --pred_len {pred_len} \
              --e_layers 2 \
              --d_layers 1 \
              --factor 5 \
              --enc_in 3 \
              --dec_in 3 \
              --c_out 3 \
              --des 'Exp' \
              --train_epochs 10 \
              --patience 3 \
              --overlapping_windows \
              --inverse \
              --scaler_type standard \
              --loss_fnc "{loss}" \
              --itr {itr} --batch_size 32 --learning_rate "{lr}"
            """

            # Run the command and capture the output
            process = subprocess.Popen(command, shell=True, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True)

            # Capture the output in real-time
            output = []
            for line in process.stdout:
                output.append(line)
                print(line, end='')  # Print in the .ipynb cell
                log_file.write(line)  # Write to the log file

            # Wait for the process to complete
            process.wait()

            # Delete the checkpoints folder and all its contents
            shutil.rmtree('./checkpoints' )

            # Extract metrics for each iteration
            iteration_metrics_scaled = extract_metrics_from_output(output, itr)
            iteration_metrics_unscaled = extract_metrics_from_output(output, itr, if_scaled=False)

            # Log the extracted metrics and save them
            for iteration, (scaled_metrics, unscaled_metrics) in enumerate(zip(iteration_metrics_scaled, iteration_metrics_unscaled), start=1):
                log_file.write(f"\nExtracted Metrics for {country}, pred_len={pred_len}, iteration={iteration}:\n")
                log_file.write(f"Scaled Metrics - MSE: {scaled_metrics[0]}, RMSE: {scaled_metrics[1]}, MAE: {scaled_metrics[2]}, RSE: {scaled_metrics[3]}\n")
                log_file.write(f"Unscaled Metrics - MSE: {unscaled_metrics[0]}, RMSE: {unscaled_metrics[1]}, MAE: {unscaled_metrics[2]}, RSE: {unscaled_metrics[3]}\n")

                # Append the results to the informer_results lists
                metrics_data = [(patchtst_results_scaled, scaled_metrics), (patchtst_results_unscaled, unscaled_metrics)]

                for result_list, metrics in metrics_data:
                    result_list.append({
                        'Loss_function': loss,
                        'Pred_len': pred_len,
                        'Iteration': iteration,
                        'MSE': metrics[0],
                        'RMSE': metrics[1],
                        'MAE': metrics[2],
                        'RSE': metrics[3]
                    })


=== Starting experiments for loss function: MSE ===


=== Starting experiments for pred_len: 24 ===

Args in experiment:
Namespace(random_seed=2021, is_training=1, model_id='IT_512_24_loss_choice_for_IT', model='PatchTST', data='custom', root_path='/vol/fob-vol3/nebenf24/riabchuv/my_work/datasets/', data_path='IT_data.csv', features='M', target='OT', freq='h', checkpoints='./checkpoints/', overlapping_windows=True, scaler_type='standard', seq_len=512, label_len=5, pred_len=24, inverse=True, loss_fnc='MSE', if_relu=False, fc_dropout=0.05, head_dropout=0.0, patch_len=16, stride=8, padding_patch='end', revin=1, affine=0, subtract_last=0, decomposition=0, kernel_size=25, individual=0, embed_type=0, enc_in=3, dec_in=3, c_out=3, d_model=512, n_heads=8, e_layers=2, d_layers=1, d_ff=2048, moving_avg=25, factor=5, distil=True, dropout=0.05, embed='timeF', activation='gelu', output_attention=False, do_predict=False, num_workers=10, itr=2, train_epochs=10, batch_size=32, patience=3, learning_rat

In [10]:
path_dir = './dataset_results'
csv_name_scaled = 'patchtst_loss_functions_results_scaled_IT.csv'
csv_name_unscaled = 'patchtst_loss_functions_results_unscaled_IT.csv'


# Convert the results into a DataFrame and save as CSV
patchtst_df_scaled = convert_results_into_df(patchtst_results_scaled, path_dir, csv_name_scaled)
patchtst_df_unscaled = convert_results_into_df(patchtst_results_unscaled, path_dir, csv_name_unscaled)
patchtst_df_scaled.round(4)

MSE    RMSE     MAE     RSE
Loss_function Iteration Pred_len                                
MSE           1         24        0.2163  0.4650  0.2981  0.4259
              2         24        0.2257  0.4751  0.3017  0.4351
              1         96        0.3643  0.6036  0.3992  0.5526
              2         96        0.3615  0.6012  0.3974  0.5505
              1         168       0.3864  0.6216  0.4178  0.5693
              2         168       0.3866  0.6218  0.4179  0.5695
RMSE          1         24        0.2188  0.4678  0.3057  0.4284
              2         24        0.2228  0.4720  0.2965  0.4323
              1         96        0.3633  0.6027  0.3975  0.5519
              2         96        0.3603  0.6002  0.3956  0.5496
              1         168       0.3849  0.6204  0.4157  0.5682
              2         168       0.3846  0.6202  0.4157  0.5680
MAE           1         24        0.2347  0.4844  0.3008  0.4436
              2         24        0.2179  0.4668  0.2705  0.4275
              1         96        0.3723  0.6102  0.3903  0.5587
              2         96        0.3720  0.6099  0.3905  0.5585
              1         168       0.3949  0.6284  0.4102  0.5755
              2         168       0.3947  0.6283  0.4102  0.5754

In [11]:
patchtst_df_unscaled.round(4)

MSE       RMSE        MAE     RSE
Loss_function Iteration Pred_len                                           
MSE           1         24        1452499.000  1205.1967   853.6146  0.0847
              2         24        1513613.875  1230.2902   866.8263  0.0865
              1         96        3107539.250  1762.8214  1208.1465  0.1241
              2         96        3066246.250  1751.0701  1197.7284  0.1232
              1         168       3617150.250  1901.8807  1296.1769  0.1340
              2         168       3511273.500  1873.8392  1283.4158  0.1320
RMSE          1         24        1550383.250  1245.1439   886.2724  0.0875
              2         24        1480387.625  1216.7118   848.0317  0.0855
              1         96        3056700.250  1748.3422  1198.0056  0.1230
              2         96        3011115.000  1735.2565  1186.3693  0.1221
              1         168       3557823.250  1886.2194  1284.5880  0.1329
              2         168       3453479.000  1858.3539  1271.8232  0.1309
MAE           1         24        1737740.625  1318.2339   870.5349  0.0926
              2         24        1284505.750  1133.3604   733.4088  0.0796
              1         96        2971120.250  1723.6937  1153.3737  0.1213
              2         96        2962697.000  1721.2487  1151.7640  0.1211
              1         168       3482605.250  1866.1740  1247.3425  0.1315
              2         168       3453181.500  1858.2738  1243.2384  0.1309

In [12]:
# Uncomment the following lines if you want to read saved results
#path_dir = './dataset_results'
#csv_name_scaled = 'patchtst_loss_functions_results_scaled.csv'
#csv_name_unscaled = 'patchtst_loss_functions_results_unscaled.csv'

# Average the iterations
ptst_scaled = pd.read_csv(os.path.join(path_dir, csv_name_scaled))
ptst_unscaled = pd.read_csv(os.path.join(path_dir, csv_name_unscaled))

ptst_res_scaled = ptst_scaled.groupby(['Pred_len', 'Loss_function']).mean().drop('Iteration', axis=1)
ptst_res_scaled.round(4)

MSE    RMSE     MAE     RSE
Pred_len Loss_function                                
24       MAE            0.2263  0.4756  0.2857  0.4356
         MSE            0.2210  0.4701  0.2999  0.4305
         RMSE           0.2208  0.4699  0.3011  0.4303
96       MAE            0.3722  0.6100  0.3904  0.5586
         MSE            0.3629  0.6024  0.3983  0.5516
         RMSE           0.3618  0.6015  0.3965  0.5507
168      MAE            0.3948  0.6283  0.4102  0.5755
         MSE            0.3865  0.6217  0.4178  0.5694
         RMSE           0.3848  0.6203  0.4157  0.5681

In [13]:
ptst_res_unscaled = ptst_unscaled.groupby(['Pred_len', 'Loss_function']).mean().drop('Iteration', axis=1)
ptst_res_unscaled.round(4)

MSE       RMSE        MAE     RSE
Pred_len Loss_function                                            
24       MAE            1.511123e+06  1225.7971   801.9718  0.0861
         MSE            1.483056e+06  1217.7434   860.2204  0.0856
         RMSE           1.515385e+06  1230.9279   867.1521  0.0865
96       MAE            2.966909e+06  1722.4712  1152.5688  0.1212
         MSE            3.086893e+06  1756.9457  1202.9374  0.1236
         RMSE           3.033908e+06  1741.7993  1192.1874  0.1226
168      MAE            3.467893e+06  1862.2239  1245.2905  0.1312
         MSE            3.564212e+06  1887.8600  1289.7963  0.1330
         RMSE           3.505651e+06  1872.2866  1278.2056  0.1319

In [14]:
shutil.rmtree("results_loss_scaled") # we do not need this directory and results anymore. If you need - comment this line

# Rename folder
os.rename("results_loss_unscaled", 'standard_unscaled')

# 3. MinMax Scaler (0, 1) Informer

We can use now "ReLU" activation function due to MinMax Scaler.

With BS 1036, ReLU - results are bad. (as twice as bad as with 32!)

In [15]:
# Paths to files and data
data_path = os.getcwd() + "/datasets/"

script_path = "./PatchTST-main/PatchTST_supervised/run_longExp.py"

# Arguments that will be used also for file names
model = "Informer"
dataset = 'IT_data.csv'
losses = ["MSE", "RMSE", "MAE"]
country = dataset[:2]

log_dir = f"logs/loss_choice/min_max_0_1_relu"
if not os.path.exists(log_dir):
    os.makedirs(log_dir)

log_file_path = f"{log_dir}/{model}_{country}.log"

In [16]:
# Dynamic variables
pred_lens = ["24", "96", "168"]
seq_len = "96"
lr = "0.0001"
itr = 2  
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

# Lists to store the results
informer_results_scaled, informer_results_unscaled = [], []

# Log file
with open(log_file_path, "w") as log_file:
    for loss in losses:
        statement_1 = f"\n=== Starting experiments for loss function: {loss} ===\n"
        log_file.write(statement_1)
        print(statement_1)  # Print to .ipynb output cell

        for pred_len in pred_lens:
            statement_2 = f"\n=== Starting experiments for pred_len: {pred_len} ===\n"
            log_file.write(statement_2)
            print(statement_2) 
            model_id = f"{country}_{seq_len}_{pred_len}_loss_choice_for_{country}"

            # Arguments for the command
            command = f"""
            python {script_path} \
              --random_seed 2021 \
              --is_training 1 \
              --root_path "{data_path}" \
              --data_path "{dataset}" \
              --model_id {model_id} \
              --model "{model}" \
              --data "custom" \
              --features M \
              --seq_len {seq_len} \
              --label_len 5 \
              --pred_len {pred_len} \
              --e_layers 2 \
              --d_layers 1 \
              --factor 5 \
              --enc_in 3 \
              --dec_in 3 \
              --c_out 3 \
              --des 'Exp' \
              --train_epochs 10 \
              --patience 3 \
              --overlapping_windows \
              --inverse \
              --scaler_type minmax \
              --if_relu \
              --activation relu \
              --loss_fnc "{loss}" \
              --itr {itr} --batch_size 32 --learning_rate "{lr}"
            """

            # Run the command and capture the output
            process = subprocess.Popen(command, shell=True, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True)

            # Capture the output in real-time
            output = []
            for line in process.stdout:
                output.append(line)
                print(line, end='')  # Print in the .ipynb cell
                log_file.write(line)  # Write to the log file

            # Wait for the process to complete
            process.wait()
            
            # Delete the checkpoints folder and all its contents
            shutil.rmtree('./checkpoints' )

            # Extract metrics for each iteration
            iteration_metrics = extract_metrics_from_output(output, itr)
            iteration_metrics_unscaled = extract_metrics_from_output(output, itr, if_scaled=False)

            # Log the extracted metrics and save them
            for iteration, (scaled_metrics, unscaled_metrics) in enumerate(zip(iteration_metrics, iteration_metrics_unscaled), start=1):
                log_file.write(f"\nExtracted Metrics for {country}, pred_len={pred_len}, iteration={iteration}:\n")
                log_file.write(f"Scaled Metrics - MSE: {scaled_metrics[0]}, RMSE: {scaled_metrics[1]}, MAE: {scaled_metrics[2]}, RSE: {scaled_metrics[3]}\n")
                log_file.write(f"Unscaled Metrics - MSE: {unscaled_metrics[0]}, RMSE: {unscaled_metrics[1]}, MAE: {unscaled_metrics[2]}, RSE: {unscaled_metrics[3]}\n")

                # Append the results to the informer_results lists
                metrics_data = [(informer_results_scaled, scaled_metrics), (informer_results_unscaled, unscaled_metrics)]

                for result_list, metrics in metrics_data:
                    result_list.append({
                        'Loss_function': loss,
                        'Pred_len': pred_len,
                        'Iteration': iteration,
                        'MSE': metrics[0],
                        'RMSE': metrics[1],
                        'MAE': metrics[2],
                        'RSE': metrics[3]
                    })


=== Starting experiments for loss function: MSE ===


=== Starting experiments for pred_len: 24 ===

Args in experiment:
Namespace(random_seed=2021, is_training=1, model_id='IT_96_24_loss_choice_for_IT', model='Informer', data='custom', root_path='/vol/fob-vol3/nebenf24/riabchuv/my_work/datasets/', data_path='IT_data.csv', features='M', target='OT', freq='h', checkpoints='./checkpoints/', overlapping_windows=True, scaler_type='minmax', seq_len=96, label_len=5, pred_len=24, inverse=True, loss_fnc='MSE', if_relu=True, fc_dropout=0.05, head_dropout=0.0, patch_len=16, stride=8, padding_patch='end', revin=1, affine=0, subtract_last=0, decomposition=0, kernel_size=25, individual=0, embed_type=0, enc_in=3, dec_in=3, c_out=3, d_model=512, n_heads=8, e_layers=2, d_layers=1, d_ff=2048, moving_avg=25, factor=5, distil=True, dropout=0.05, embed='timeF', activation='relu', output_attention=False, do_predict=False, num_workers=10, itr=2, train_epochs=10, batch_size=32, patience=3, learning_rate=0.0

In [17]:
path_dir = './dataset_results'
csv_name_scaled = 'informer_loss_functions_results_scaled_minmax_0_1_relu_IT.csv'
csv_name_unscaled = 'informer_loss_functions_results_unscaled_minmax_0_1_relu_IT.csv'

# Convert the results into a DataFrame and save as CSV
informer_df_scaled = convert_results_into_df(informer_results_scaled, path_dir, csv_name_scaled)
informer_df_unscaled = convert_results_into_df(informer_results_unscaled, path_dir, csv_name_unscaled)
informer_df_scaled.round(4)

MSE    RMSE     MAE     RSE
Loss_function Iteration Pred_len                                
MSE           1         24        0.0108  0.1040  0.0631  0.3932
              2         24        0.0109  0.1044  0.0650  0.3946
              1         96        0.0192  0.1386  0.0892  0.5240
              2         96        0.0179  0.1339  0.0864  0.5064
              1         168       0.0192  0.1386  0.0919  0.5244
              2         168       0.0193  0.1388  0.0903  0.5252
RMSE          1         24        0.0107  0.1036  0.0632  0.3915
              2         24        0.0114  0.1070  0.0662  0.4042
              1         96        0.0180  0.1340  0.0880  0.5066
              2         96        0.0187  0.1366  0.0893  0.5167
              1         168       0.0193  0.1389  0.0917  0.5255
              2         168       0.0191  0.1381  0.0900  0.5224
MAE           1         24        0.0678  0.2604  0.1374  0.9841
              2         24        0.0116  0.1076  0.0625  0.4065
              1         96        0.0759  0.2755  0.1601  1.0418
              2         96        0.0756  0.2749  0.1590  1.0395
              1         168       0.0212  0.1455  0.0927  0.5506
              2         168       0.0227  0.1508  0.0952  0.5704

In [18]:
informer_df_unscaled.round(4)

MSE       RMSE        MAE     RSE
Loss_function Iteration Pred_len                                           
MSE           1         24        1590108.375  1260.9950   816.7712  0.0886
              2         24        1765169.375  1328.5968   861.3684  0.0934
              1         96        3753499.000  1937.3949  1257.5044  0.1363
              2         96        3062977.000  1750.1362  1161.6783  0.1232
              1         168       3780508.000  1944.3529  1279.1031  0.1370
              2         168       3504085.500  1871.9203  1241.2753  0.1319
RMSE          1         24        1665273.250  1290.4547   826.0386  0.0907
              2         24        2052062.125  1432.5021   900.5717  0.1007
              1         96        3025316.500  1739.3438  1175.1943  0.1224
              2         96        3516699.000  1875.2864  1229.0858  0.1320
              1         168       3779750.750  1944.1581  1275.7247  0.1369
              2         168       3414384.250  1847.8053  1228.9226  0.1302
MAE           1         24        7994465.000  2827.4485  1573.5731  0.1987
              2         24        1621922.000  1273.5470   778.6210  0.0895
              1         96        9655030.000  3107.2544  1923.6842  0.2187
              2         96        9176649.000  3029.2983  1877.2694  0.2132
              1         168       3859127.000  1964.4661  1258.3188  0.1384
              2         168       4104890.000  2026.0529  1282.5724  0.1427

In [19]:
# Uncomment the following lines if you want to read saved results
#path_dir = './dataset_results'
#csv_name_scaled = 'informer_loss_functions_results_scaled_minmax_0_1_relu.csv'
#csv_name_unscaled = 'informer_loss_functions_results_unscaled_minmax_0_1_relu.csv'

# Average the iterations
informer_scaled = pd.read_csv(os.path.join(path_dir, csv_name_scaled))
informer_unscaled = pd.read_csv(os.path.join(path_dir, csv_name_unscaled))

inf_res_scaled = informer_scaled.groupby(['Pred_len', 'Loss_function']).mean().drop('Iteration', axis=1)
inf_res_unscaled = informer_unscaled.groupby(['Pred_len', 'Loss_function']).mean().sort_index().drop('Iteration', axis=1)
inf_res_scaled.round(4)

MSE    RMSE     MAE     RSE
Pred_len Loss_function                                
24       MAE            0.0397  0.1840  0.1000  0.6953
         MSE            0.0109  0.1042  0.0640  0.3939
         RMSE           0.0111  0.1053  0.0647  0.3979
96       MAE            0.0758  0.2752  0.1595  1.0407
         MSE            0.0186  0.1363  0.0878  0.5152
         RMSE           0.0183  0.1353  0.0887  0.5116
168      MAE            0.0220  0.1481  0.0940  0.5605
         MSE            0.0192  0.1387  0.0911  0.5248
         RMSE           0.0192  0.1385  0.0908  0.5239

In [20]:
inf_res_unscaled.round(4)

MSE       RMSE        MAE     RSE
Pred_len Loss_function                                            
24       MAE            4.808194e+06  2050.4977  1176.0971  0.1441
         MSE            1.677639e+06  1294.7959   839.0698  0.0910
         RMSE           1.858668e+06  1361.4784   863.3051  0.0957
96       MAE            9.415840e+06  3068.2764  1900.4768  0.2159
         MSE            3.408238e+06  1843.7656  1209.5914  0.1298
         RMSE           3.271008e+06  1807.3151  1202.1401  0.1272
168      MAE            3.982008e+06  1995.2595  1270.4456  0.1405
         MSE            3.642297e+06  1908.1366  1260.1892  0.1344
         RMSE           3.597068e+06  1895.9817  1252.3237  0.1336

# 4. MinMax Scaler (0, 1) PatchTST

In [21]:
log_dir = f"logs/loss_choice/min_max_0_1_relu"
if not os.path.exists(log_dir):
    os.makedirs(log_dir)

In [22]:
# Dynamic variables
pred_lens = ["24", "96", "168"]
seq_len = "512"
lr = "0.0001"
model = "PatchTST"
itr = 2  
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

# New log file path
log_file_path = f"{log_dir}/{model}_{country}.log"

patchtst_results_scaled, patchtst_results_unscaled = [], []

with open(log_file_path, "w") as log_file:
    for loss in losses:
        statement_1 = f"\n=== Starting experiments for loss function: {loss} ===\n"
        log_file.write(statement_1)
        print(statement_1)  # Print to notebook

        for pred_len in pred_lens:
            statement_2 = f"\n=== Starting experiments for pred_len: {pred_len} ===\n"
            log_file.write(statement_2)
            print(statement_2) 
            model_id = f"{country}_{seq_len}_{pred_len}_loss_choice_for_{country}"

            # Command arguments
            command = f"""
            python {script_path} \
              --random_seed 2021 \
              --is_training 1 \
              --root_path "{data_path}" \
              --data_path "{dataset}" \
              --model_id {model_id} \
              --model "{model}" \
              --data "custom" \
              --features M \
              --seq_len {seq_len} \
              --label_len 5 \
              --pred_len {pred_len} \
              --e_layers 2 \
              --d_layers 1 \
              --factor 5 \
              --enc_in 3 \
              --dec_in 3 \
              --c_out 3 \
              --des 'Exp' \
              --train_epochs 10 \
              --patience 3 \
              --overlapping_windows \
              --inverse \
              --scaler_type minmax \
              --if_relu \
              --loss_fnc "{loss}" \
              --itr {itr} --batch_size 32 --learning_rate "{lr}"
            """

            # Run the command and capture the output
            process = subprocess.Popen(command, shell=True, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True)

            # Capture the output in real-time
            output = []
            for line in process.stdout:
                output.append(line)
                print(line, end='')  # Print in the .ipynb cell
                log_file.write(line)  # Write to the log file

            # Wait for the process to complete
            process.wait()

            # Delete the checkpoints folder and all its contents
            shutil.rmtree('./checkpoints' )

            # Extract metrics for each iteration
            iteration_metrics_scaled = extract_metrics_from_output(output, itr)
            iteration_metrics_unscaled = extract_metrics_from_output(output, itr, if_scaled=False)

            # Log the extracted metrics and save them
            for iteration, (scaled_metrics, unscaled_metrics) in enumerate(zip(iteration_metrics_scaled, iteration_metrics_unscaled), start=1):
                log_file.write(f"\nExtracted Metrics for {country}, pred_len={pred_len}, iteration={iteration}:\n")
                log_file.write(f"Scaled Metrics - MSE: {scaled_metrics[0]}, RMSE: {scaled_metrics[1]}, MAE: {scaled_metrics[2]}, RSE: {scaled_metrics[3]}\n")
                log_file.write(f"Unscaled Metrics - MSE: {unscaled_metrics[0]}, RMSE: {unscaled_metrics[1]}, MAE: {unscaled_metrics[2]}, RSE: {unscaled_metrics[3]}\n")

                # Append the results to the informer_results lists
                metrics_data = [(patchtst_results_scaled, scaled_metrics), (patchtst_results_unscaled, unscaled_metrics)]

                for result_list, metrics in metrics_data:
                    result_list.append({
                        'Loss_function': loss,
                        'Pred_len': pred_len,
                        'Iteration': iteration,
                        'MSE': metrics[0],
                        'RMSE': metrics[1],
                        'MAE': metrics[2],
                        'RSE': metrics[3]
                    })


=== Starting experiments for loss function: MSE ===


=== Starting experiments for pred_len: 24 ===

Args in experiment:
Namespace(random_seed=2021, is_training=1, model_id='IT_512_24_loss_choice_for_IT', model='PatchTST', data='custom', root_path='/vol/fob-vol3/nebenf24/riabchuv/my_work/datasets/', data_path='IT_data.csv', features='M', target='OT', freq='h', checkpoints='./checkpoints/', overlapping_windows=True, scaler_type='minmax', seq_len=512, label_len=5, pred_len=24, inverse=True, loss_fnc='MSE', if_relu=True, fc_dropout=0.05, head_dropout=0.0, patch_len=16, stride=8, padding_patch='end', revin=1, affine=0, subtract_last=0, decomposition=0, kernel_size=25, individual=0, embed_type=0, enc_in=3, dec_in=3, c_out=3, d_model=512, n_heads=8, e_layers=2, d_layers=1, d_ff=2048, moving_avg=25, factor=5, distil=True, dropout=0.05, embed='timeF', activation='gelu', output_attention=False, do_predict=False, num_workers=10, itr=2, train_epochs=10, batch_size=32, patience=3, learning_rate=0

In [23]:
path_dir = './dataset_results'
csv_name_scaled = 'patchtst_loss_functions_results_scaled_minmax_0_1_relu_IT.csv'
csv_name_unscaled = 'patchtst_loss_functions_results_unscaled_minmax_0_1_relu_IT.csv'


# Convert the results into a DataFrame and save as CSV
patchtst_df_scaled = convert_results_into_df(patchtst_results_scaled, path_dir, csv_name_scaled)
patchtst_df_unscaled = convert_results_into_df(patchtst_results_unscaled, path_dir, csv_name_unscaled)
#patchtst_df_scaled = pd.read_csv(os.path.join(path_dir, csv_name_scaled))
patchtst_df_scaled.round(4)

MSE    RMSE     MAE     RSE
Loss_function Iteration Pred_len                                
MSE           1         24        0.0114  0.1067  0.0656  0.4030
              2         24        0.0115  0.1074  0.0653  0.4058
              1         96        0.0182  0.1349  0.0871  0.5099
              2         96        0.0181  0.1345  0.0868  0.5084
              1         168       0.0194  0.1392  0.0916  0.5265
              2         168       0.0194  0.1392  0.0913  0.5267
RMSE          1         24        0.0114  0.1068  0.0658  0.4034
              2         24        0.0115  0.1071  0.0665  0.4048
              1         96        0.0181  0.1346  0.0867  0.5091
              2         96        0.0180  0.1342  0.0863  0.5074
              1         168       0.0193  0.1389  0.0912  0.5255
              2         168       0.0193  0.1388  0.0907  0.5252
MAE           1         24        0.0103  0.1014  0.0584  0.3833
              2         24        0.0106  0.1030  0.0597  0.3892
              1         96        0.0214  0.1464  0.0866  0.5535
              2         96        0.0203  0.1424  0.0852  0.5384
              1         168       0.0198  0.1406  0.0900  0.5321
              2         168       0.0198  0.1407  0.0900  0.5325

In [24]:
patchtst_results_scaled = pd.read_csv(os.path.join(path_dir, csv_name_scaled))
patchtst_df_unscaled.round(4)

MSE       RMSE        MAE     RSE
Loss_function Iteration Pred_len                                           
MSE           1         24        1649581.250  1284.3602   850.4399  0.0903
              2         24        1637449.875  1279.6288   845.5874  0.0899
              1         96        3181651.500  1783.7185  1182.0157  0.1255
              2         96        3139555.500  1771.8790  1171.6736  0.1247
              1         168       3698332.250  1923.1049  1273.1566  0.1355
              2         168       3587113.500  1893.9677  1255.8635  0.1334
RMSE          1         24        1851359.875  1360.6469   874.3544  0.0956
              2         24        1904325.375  1379.9730   891.1001  0.0970
              1         96        3113013.750  1764.3734  1168.9977  0.1242
              2         96        3066430.250  1751.1226  1157.3804  0.1232
              1         168       3630107.250  1905.2841  1260.4178  0.1342
              2         168       3512351.750  1874.1270  1241.5947  0.1320
MAE           1         24        1229446.000  1108.8038   701.6744  0.0779
              2         24        1335150.250  1155.4871   726.6998  0.0812
              1         96        2624052.500  1619.8927  1041.3607  0.1140
              2         96        2622210.250  1619.3240  1035.5718  0.1140
              1         168       3422754.750  1850.0688  1207.7421  0.1303
              2         168       3396051.750  1842.8380  1203.2321  0.1298

In [25]:
# Uncomment the following lines if you want to read saved results
#path_dir = './dataset_results'
#csv_name_scaled = 'patchtst_loss_functions_results_scaled_minmax_0_1_relu.csv'
#csv_name_unscaled = 'patchtst_loss_functions_results_unscaled_minmax_0_1_relu.csv'

# Average the iterations
ptst_scaled = pd.read_csv(os.path.join(path_dir, csv_name_scaled))
ptst_unscaled = pd.read_csv(os.path.join(path_dir, csv_name_unscaled))

ptst_res_scaled = ptst_scaled.groupby(['Pred_len', 'Loss_function']).mean().drop('Iteration', axis=1)
ptst_res_scaled.round(4)

MSE    RMSE     MAE     RSE
Pred_len Loss_function                                
24       MAE            0.0104  0.1022  0.0591  0.3862
         MSE            0.0115  0.1070  0.0655  0.4044
         RMSE           0.0114  0.1069  0.0661  0.4041
96       MAE            0.0209  0.1444  0.0859  0.5460
         MSE            0.0181  0.1347  0.0870  0.5092
         RMSE           0.0181  0.1344  0.0865  0.5083
168      MAE            0.0198  0.1407  0.0900  0.5323
         MSE            0.0194  0.1392  0.0915  0.5266
         RMSE           0.0193  0.1389  0.0910  0.5254

In [26]:
ptst_res_unscaled = ptst_unscaled.groupby(['Pred_len', 'Loss_function']).mean().drop('Iteration', axis=1)
ptst_res_unscaled.round(4)

MSE       RMSE        MAE     RSE
Pred_len Loss_function                                            
24       MAE            1.282298e+06  1132.1454   714.1871  0.0796
         MSE            1.643516e+06  1281.9945   848.0136  0.0901
         RMSE           1.877843e+06  1370.3099   882.7272  0.0963
96       MAE            2.623131e+06  1619.6083  1038.4662  0.1140
         MSE            3.160604e+06  1777.7988  1176.8447  0.1251
         RMSE           3.089722e+06  1757.7480  1163.1890  0.1237
168      MAE            3.409403e+06  1846.4534  1205.4871  0.1301
         MSE            3.642723e+06  1908.5363  1264.5101  0.1344
         RMSE           3.571230e+06  1889.7055  1251.0063  0.1331

In [27]:
# Rename folders
new_path_name = 'minmax_0_1_relu_unscaled'
shutil.rmtree("results_loss_scaled") # we do not need this directory and results anymore. If you need - comment this line
os.rename("results_loss_unscaled", new_path_name)

# 5. MinMax Scaler (0, 5) Informer


In [2]:
# Paths to files and data
data_path = os.getcwd() + "/datasets/"

script_path = "./PatchTST-main/PatchTST_supervised/run_longExp.py"

# Arguments that will be used also for file names
model = "Informer"
dataset = 'IT_data.csv'
losses = ["MSE", "RMSE", "MAE"]
country = dataset[:2]

log_dir = f"logs/loss_choice/min_max_0_5_relu"
if not os.path.exists(log_dir):
    os.makedirs(log_dir)

log_file_path = f"{log_dir}/{model}_{country}.log"

In [4]:
# Dynamic variables
pred_lens = ["24", "96", "168"]
seq_len = "96"
lr = "0.0001"
itr = 2  
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

# Lists to store the results
informer_results_scaled, informer_results_unscaled = [], []

# Log file
with open(log_file_path, "w") as log_file:
    for loss in losses:
        statement_1 = f"\n=== Starting experiments for loss function: {loss} ===\n"
        log_file.write(statement_1)
        print(statement_1)  # Print to .ipynb output cell

        for pred_len in pred_lens:
            statement_2 = f"\n=== Starting experiments for pred_len: {pred_len} ===\n"
            log_file.write(statement_2)
            print(statement_2) 
            model_id = f"{country}_{seq_len}_{pred_len}_loss_choice_for_{country}"

            # Arguments for the command
            command = f"""
            python {script_path} \
              --random_seed 2021 \
              --is_training 1 \
              --root_path "{data_path}" \
              --data_path "{dataset}" \
              --model_id {model_id} \
              --model "{model}" \
              --data "custom" \
              --features M \
              --seq_len {seq_len} \
              --label_len 5 \
              --pred_len {pred_len} \
              --e_layers 2 \
              --d_layers 1 \
              --factor 5 \
              --enc_in 3 \
              --dec_in 3 \
              --c_out 3 \
              --des 'Exp' \
              --train_epochs 10 \
              --patience 3 \
              --overlapping_windows \
              --inverse \
              --loss_fnc "{loss}" \
              --scaler_type minmax2 \
              --if_relu \
              --activation relu \
              --itr {itr} --batch_size 32 --learning_rate "{lr}"
            """

            # Run the command and capture the output
            process = subprocess.Popen(command, shell=True, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True)

            # Capture the output in real-time
            output = []
            for line in process.stdout:
                output.append(line)
                print(line, end='')  # Print in the .ipynb cell
                log_file.write(line)  # Write to the log file

            # Wait for the process to complete
            process.wait()
            
            # Delete the checkpoints folder and all its contents
            shutil.rmtree('./checkpoints' )

            # Extract metrics for each iteration
            iteration_metrics = extract_metrics_from_output(output, itr)
            iteration_metrics_unscaled = extract_metrics_from_output(output, itr, if_scaled=False)

            # Log the extracted metrics and save them
            for iteration, (scaled_metrics, unscaled_metrics) in enumerate(zip(iteration_metrics, iteration_metrics_unscaled), start=1):
                log_file.write(f"\nExtracted Metrics for {country}, pred_len={pred_len}, iteration={iteration}:\n")
                log_file.write(f"Scaled Metrics - MSE: {scaled_metrics[0]}, RMSE: {scaled_metrics[1]}, MAE: {scaled_metrics[2]}, RSE: {scaled_metrics[3]}\n")
                log_file.write(f"Unscaled Metrics - MSE: {unscaled_metrics[0]}, RMSE: {unscaled_metrics[1]}, MAE: {unscaled_metrics[2]}, RSE: {unscaled_metrics[3]}\n")

                # Append the results to the informer_results lists
                metrics_data = [(informer_results_scaled, scaled_metrics), (informer_results_unscaled, unscaled_metrics)]

                for result_list, metrics in metrics_data:
                    result_list.append({
                        'Loss_function': loss,
                        'Pred_len': pred_len,
                        'Iteration': iteration,
                        'MSE': metrics[0],
                        'RMSE': metrics[1],
                        'MAE': metrics[2],
                        'RSE': metrics[3]
                    })


=== Starting experiments for loss function: MSE ===


=== Starting experiments for pred_len: 24 ===

Args in experiment:
Namespace(random_seed=2021, is_training=1, model_id='DE_96_24_loss_choice_for_DE', model='Informer', data='custom', root_path='/vol/cs-hu/riabchuv/my_work/datasets/', data_path='DE_data.csv', features='M', target='OT', freq='h', checkpoints='./checkpoints/', overlapping_windows=True, seq_len=96, label_len=5, pred_len=24, inverse=True, loss_fnc='MSE', fc_dropout=0.05, head_dropout=0.0, patch_len=16, stride=8, padding_patch='end', revin=1, affine=0, subtract_last=0, decomposition=0, kernel_size=25, individual=0, embed_type=0, enc_in=5, dec_in=5, c_out=5, d_model=512, n_heads=8, e_layers=2, d_layers=1, d_ff=2048, moving_avg=25, factor=5, distil=True, dropout=0.05, embed='timeF', activation='relu', output_attention=False, do_predict=False, num_workers=10, itr=2, train_epochs=10, batch_size=32, patience=3, learning_rate=0.0001, des='Exp', lradj='type3', pct_start=0.3, us

In [5]:
path_dir = './dataset_results'
csv_name_scaled = 'informer_loss_functions_results_scaled_minmax_0_5_relu_IT.csv'
csv_name_unscaled = 'informer_loss_functions_results_unscaled_minmax_0_5_relu_IT.csv'

# Convert the results into a DataFrame and save as CSV
informer_df_scaled = convert_results_into_df(informer_results_scaled, path_dir, csv_name_scaled)
informer_df_unscaled = convert_results_into_df(informer_results_unscaled, path_dir, csv_name_unscaled)
informer_df_scaled.round(4)

MSE    RMSE     MAE     RSE
Loss_function Iteration Pred_len                                
MSE           1         24        0.5900  0.7681  0.5134  0.5425
              2         24        0.5565  0.7460  0.5017  0.5269
              1         96        0.9737  0.9868  0.7140  0.6989
              2         96        1.0281  1.0139  0.7083  0.7181
              1         168       1.0188  1.0093  0.7099  0.7152
              2         168       1.0490  1.0242  0.7395  0.7257
RMSE          1         24        0.5771  0.7597  0.5050  0.5365
              2         24        0.5795  0.7613  0.5110  0.5377
              1         96        0.9812  0.9905  0.7067  0.7015
              2         96        1.0184  1.0091  0.7146  0.7147
              1         168       1.0020  1.0010  0.7067  0.7093
              2         168       1.1629  1.0784  0.7698  0.7641
MAE           1         24        0.6073  0.7793  0.4910  0.5504
              2         24        0.5877  0.7666  0.4944  0.5415
              1         96        0.9911  0.9955  0.6899  0.7051
              2         96        1.0454  1.0224  0.7026  0.7241
              1         168       1.0631  1.0311  0.7205  0.7306
              2         168       1.0521  1.0257  0.7103  0.7268

In [6]:
informer_df_unscaled.round(4)

MSE       RMSE        MAE     RSE
Loss_function Iteration Pred_len                                          
MSE           1         24        20480482.0  4525.5366  2909.4851  0.2250
              2         24        18213778.0  4267.7603  2791.6816  0.2122
              1         96        35709004.0  5975.7012  4075.1108  0.2976
              2         96        36943168.0  6078.0894  3976.2830  0.3027
              1         168       36735616.0  6060.9912  3992.3479  0.3020
              2         168       38725652.0  6222.9937  4226.7588  0.3101
RMSE          1         24        19703646.0  4438.8789  2844.3523  0.2207
              2         24        19322682.0  4395.7573  2863.1350  0.2186
              1         96        35458704.0  5954.7212  4002.4402  0.2965
              2         96        36999436.0  6082.7163  4040.1528  0.3029
              1         168       36233816.0  6019.4531  3968.3572  0.2999
              2         168       45239896.0  6726.0610  4444.7422  0.3351
MAE           1         24        20904604.0  4572.1553  2763.4695  0.2273
              2         24        19740060.0  4442.9785  2748.1685  0.2209
              1         96        35747956.0  5978.9595  3889.9070  0.2978
              2         96        38837292.0  6231.9575  4011.7781  0.3104
              1         168       38654340.0  6217.2612  4107.8438  0.3098
              2         168       38850424.0  6233.0107  4030.0308  0.3106

In [7]:
# Uncomment the following lines if you want to read saved results
#path_dir = './dataset_results'
#csv_name_scaled = 'informer_loss_functions_results_scaled_minmax_0_5_relu.csv'
#csv_name_unscaled = 'informer_loss_functions_results_unscaled_minmax_0_5_relu.csv'

# Average the iterations
informer_scaled = pd.read_csv(os.path.join(path_dir, csv_name_scaled))
informer_unscaled = pd.read_csv(os.path.join(path_dir, csv_name_unscaled))

inf_res_scaled = informer_scaled.groupby(['Pred_len', 'Loss_function']).mean().drop('Iteration', axis=1)
inf_res_unscaled = informer_unscaled.groupby(['Pred_len', 'Loss_function']).mean().sort_index().drop('Iteration', axis=1)
inf_res_scaled.round(4)

MSE    RMSE     MAE     RSE
Pred_len Loss_function                                
24       MAE            0.5975  0.7730  0.4927  0.5460
         MSE            0.5732  0.7570  0.5075  0.5347
         RMSE           0.5783  0.7605  0.5080  0.5371
96       MAE            1.0182  1.0090  0.6962  0.7146
         MSE            1.0009  1.0003  0.7112  0.7085
         RMSE           0.9998  0.9998  0.7107  0.7081
168      MAE            1.0576  1.0284  0.7154  0.7287
         MSE            1.0339  1.0168  0.7247  0.7204
         RMSE           1.0825  1.0397  0.7382  0.7367

In [8]:
inf_res_unscaled.round(4)

MSE       RMSE        MAE     RSE
Pred_len Loss_function                                          
24       MAE            20322332.0  4507.5669  2755.8190  0.2241
         MSE            19347130.0  4396.6484  2850.5834  0.2186
         RMSE           19513164.0  4417.3181  2853.7437  0.2196
96       MAE            37292624.0  6105.4585  3950.8425  0.3041
         MSE            36326086.0  6026.8953  4025.6969  0.3001
         RMSE           36229070.0  6018.7188  4021.2965  0.2997
168      MAE            38752382.0  6225.1360  4068.9373  0.3102
         MSE            37730634.0  6141.9924  4109.5533  0.3060
         RMSE           40736856.0  6372.7571  4206.5497  0.3175

# 6. MinMax Scaler (0, 5) PatchTST

In [11]:
# Dynamic variables
pred_lens = ["24", "96", "168"]
seq_len = "512"
lr = "0.0001"
model = "PatchTST"
itr = 2  
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

# New log file path
log_file_path = f"{log_dir}/{model}_{country}.log"

patchtst_results_scaled, patchtst_results_unscaled = [], []

with open(log_file_path, "w") as log_file:
    for loss in losses:
        statement_1 = f"\n=== Starting experiments for loss function: {loss} ===\n"
        log_file.write(statement_1)
        print(statement_1)  # Print to notebook

        for pred_len in pred_lens:
            statement_2 = f"\n=== Starting experiments for pred_len: {pred_len} ===\n"
            log_file.write(statement_2)
            print(statement_2) 
            model_id = f"{country}_{seq_len}_{pred_len}_loss_choice_for_{country}"

            # Command arguments
            command = f"""
            python {script_path} \
              --random_seed 2021 \
              --is_training 1 \
              --root_path "{data_path}" \
              --data_path "{dataset}" \
              --model_id {model_id} \
              --model "{model}" \
              --data "custom" \
              --features M \
              --seq_len {seq_len} \
              --label_len 5 \
              --pred_len {pred_len} \
              --e_layers 2 \
              --d_layers 1 \
              --factor 5 \
              --enc_in 3 \
              --dec_in 3 \
              --c_out 3 \
              --des 'Exp' \
              --train_epochs 10 \
              --patience 3 \
              --overlapping_windows \
              --inverse \
              --loss_fnc "{loss}" \
              --scaler_type minmax2 \
              --if_relu \
              --itr {itr} --batch_size 32 --learning_rate "{lr}"
            """

            # Run the command and capture the output
            process = subprocess.Popen(command, shell=True, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True)

            # Capture the output in real-time
            output = []
            for line in process.stdout:
                output.append(line)
                print(line, end='')  # Print in the .ipynb cell
                log_file.write(line)  # Write to the log file

            # Wait for the process to complete
            process.wait()

            # Delete the checkpoints folder and all its contents
            shutil.rmtree('./checkpoints' )

            # Extract metrics for each iteration
            iteration_metrics_scaled = extract_metrics_from_output(output, itr)
            iteration_metrics_unscaled = extract_metrics_from_output(output, itr, if_scaled=False)

            # Log the extracted metrics and save them
            for iteration, (scaled_metrics, unscaled_metrics) in enumerate(zip(iteration_metrics_scaled, iteration_metrics_unscaled), start=1):
                log_file.write(f"\nExtracted Metrics for {country}, pred_len={pred_len}, iteration={iteration}:\n")
                log_file.write(f"Scaled Metrics - MSE: {scaled_metrics[0]}, RMSE: {scaled_metrics[1]}, MAE: {scaled_metrics[2]}, RSE: {scaled_metrics[3]}\n")
                log_file.write(f"Unscaled Metrics - MSE: {unscaled_metrics[0]}, RMSE: {unscaled_metrics[1]}, MAE: {unscaled_metrics[2]}, RSE: {unscaled_metrics[3]}\n")

                # Append the results to the informer_results lists
                metrics_data = [(patchtst_results_scaled, scaled_metrics), (patchtst_results_unscaled, unscaled_metrics)]

                for result_list, metrics in metrics_data:
                    result_list.append({
                        'Loss_function': loss,
                        'Pred_len': pred_len,
                        'Iteration': iteration,
                        'MSE': metrics[0],
                        'RMSE': metrics[1],
                        'MAE': metrics[2],
                        'RSE': metrics[3]
                    })


=== Starting experiments for loss function: MSE ===


=== Starting experiments for pred_len: 24 ===

Args in experiment:
Namespace(random_seed=2021, is_training=1, model_id='DE_512_24_loss_choice_for_DE', model='PatchTST', data='custom', root_path='/vol/fob-vol3/nebenf24/riabchuv/my_work/datasets/', data_path='DE_data.csv', features='M', target='OT', freq='h', checkpoints='./checkpoints/', overlapping_windows=True, scaler_type='minmax2', seq_len=512, label_len=5, pred_len=24, inverse=True, loss_fnc='MSE', if_relu=True, fc_dropout=0.05, head_dropout=0.0, patch_len=16, stride=8, padding_patch='end', revin=1, affine=0, subtract_last=0, decomposition=0, kernel_size=25, individual=0, embed_type=0, enc_in=5, dec_in=5, c_out=5, d_model=512, n_heads=8, e_layers=2, d_layers=1, d_ff=2048, moving_avg=25, factor=5, distil=True, dropout=0.05, embed='timeF', activation='gelu', output_attention=False, do_predict=False, num_workers=10, itr=2, train_epochs=10, batch_size=32, patience=3, learning_rate=

In [12]:
path_dir = './dataset_results'
csv_name_scaled = 'patchtst_loss_functions_results_scaled_minmax_0_5_relu_IT.csv'
csv_name_unscaled = 'patchtst_loss_functions_results_unscaled_minmax_0_5_relu_IT.csv'


# Convert the results into a DataFrame and save as CSV
patchtst_df_scaled = convert_results_into_df(patchtst_results_scaled, path_dir, csv_name_scaled)
patchtst_df_unscaled = convert_results_into_df(patchtst_results_unscaled, path_dir, csv_name_unscaled)
patchtst_df_scaled.round(4)

MSE    RMSE     MAE     RSE
Loss_function Iteration Pred_len                                
MSE           1         24        0.5360  0.7321  0.4716  0.5171
              2         24        0.5333  0.7303  0.4712  0.5158
              1         96        0.8908  0.9438  0.6565  0.6685
              2         96        0.9085  0.9531  0.6624  0.6750
              1         168       0.9642  0.9819  0.6851  0.6957
              2         168       0.9401  0.9696  0.6843  0.6870
RMSE          1         24        0.5414  0.7358  0.4735  0.5197
              2         24        0.5330  0.7301  0.4693  0.5156
              1         96        0.8934  0.9452  0.6558  0.6694
              2         96        0.8944  0.9457  0.6557  0.6698
              1         168       0.9804  0.9902  0.6853  0.7016
              2         168       0.9393  0.9691  0.6835  0.6867
MAE           1         24        0.5424  0.7365  0.4505  0.5202
              2         24        0.5394  0.7344  0.4493  0.5187
              1         96        0.8917  0.9443  0.6294  0.6688
              2         96        0.9266  0.9626  0.6355  0.6817
              1         168       0.9621  0.9809  0.6685  0.6950
              2         168       0.9993  0.9997  0.6711  0.7083

In [13]:
patchtst_df_unscaled.round(4)

MSE       RMSE        MAE     RSE
Loss_function Iteration Pred_len                                          
MSE           1         24        17142486.0  4140.3486  2565.9302  0.2059
              2         24        17089198.0  4133.9082  2575.7898  0.2055
              1         96        31524144.0  5614.6367  3636.7886  0.2796
              2         96        33323196.0  5772.6245  3707.9365  0.2875
              1         168       34983416.0  5914.6782  3824.7483  0.2947
              2         168       34048708.0  5835.1270  3839.2070  0.2907
RMSE          1         24        17373884.0  4168.1992  2577.1997  0.2073
              2         24        17012618.0  4124.6357  2558.6548  0.2051
              1         96        31708800.0  5631.0566  3635.2244  0.2804
              2         96        32409162.0  5692.9043  3661.5996  0.2835
              1         168       35351660.0  5945.7261  3821.0962  0.2962
              2         168       33994148.0  5830.4502  3832.3152  0.2905
MAE           1         24        16739734.0  4091.4221  2419.3403  0.2034
              2         24        16871196.0  4107.4561  2425.1201  0.2042
              1         96        31304240.0  5595.0190  3466.7358  0.2786
              2         96        32648818.0  5713.9146  3491.8354  0.2846
              1         168       34872248.0  5905.2729  3729.8071  0.2942
              2         168       35697720.0  5974.7568  3709.1443  0.2977

In [14]:
# Uncomment the following lines if you want to read saved results
path_dir = './dataset_results'
csv_name_scaled = 'patchtst_loss_functions_results_scaled_minmax_0_5_relu_IT.csv'
csv_name_unscaled = 'patchtst_loss_functions_results_unscaled_minmax_0_5_relu_IT.csv'

# Average the iterations
ptst_scaled = pd.read_csv(os.path.join(path_dir, csv_name_scaled))
ptst_unscaled = pd.read_csv(os.path.join(path_dir, csv_name_unscaled))

ptst_res_scaled = ptst_scaled.groupby(['Pred_len', 'Loss_function']).mean().drop('Iteration', axis=1)
ptst_res_scaled.round(4)

MSE    RMSE     MAE     RSE
Pred_len Loss_function                                
24       MAE            0.5409  0.7354  0.4499  0.5194
         MSE            0.5346  0.7312  0.4714  0.5164
         RMSE           0.5372  0.7329  0.4714  0.5177
96       MAE            0.9091  0.9534  0.6325  0.6753
         MSE            0.8996  0.9485  0.6595  0.6718
         RMSE           0.8939  0.9455  0.6558  0.6696
168      MAE            0.9807  0.9903  0.6698  0.7016
         MSE            0.9521  0.9758  0.6847  0.6914
         RMSE           0.9598  0.9797  0.6844  0.6941

In [15]:
ptst_res_unscaled = ptst_unscaled.groupby(['Pred_len', 'Loss_function']).mean().drop('Iteration', axis=1)
ptst_res_unscaled.round(4)

MSE       RMSE        MAE     RSE
Pred_len Loss_function                                          
24       MAE            16805465.0  4099.4391  2422.2302  0.2038
         MSE            17115842.0  4137.1284  2570.8600  0.2057
         RMSE           17193251.0  4146.4175  2567.9272  0.2062
96       MAE            31976529.0  5654.4668  3479.2856  0.2816
         MSE            32423670.0  5693.6306  3672.3625  0.2835
         RMSE           32058981.0  5661.9805  3648.4120  0.2820
168      MAE            35284984.0  5940.0149  3719.4757  0.2960
         MSE            34516062.0  5874.9026  3831.9777  0.2927
         RMSE           34672904.0  5888.0881  3826.7057  0.2934

In [16]:
# Rename folders
new_path_name = 'minmax_0_5_relu_unscaled'
shutil.rmtree("results_loss_scaled") # we do not need this directory and results anymore. If you need - comment this line
os.rename("results_loss_unscaled", new_path_name)